In [2]:
import numpy as np
import pandas as pd
import pickle

In [2]:
df=pd.read_csv('sampledDataset.csv')
df.head()

,Unnamed: 0,hindi,english
0,0,दो वर्ष की उम्र से स्थायी छेदक दांत पैदा होने ...,"At the age of two years, the permanent incisor..."
1,1,किसी देश में बैंकिंग व्यवस्था का ढांचा।,The organisational set up of banking in a coun...
2,2,"दो गवाह जो मारे गए उनमें से एक को तो खुलेआम, द...","Of the two that were killed, one was shot dead..."
3,3,इस लड़ाई में सारे देश में एक चेतना जागी है।,The whole country has awakened during the war.
4,4,वह बोले हम तो गुनाहगारों (क़ौमे लूत) की तरफ भे...,"They replied, ""We have been sent to a sinful p..."


In [3]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
df.head()

,hindi,english
0,दो वर्ष की उम्र से स्थायी छेदक दांत पैदा होने ...,"At the age of two years, the permanent incisor..."
1,किसी देश में बैंकिंग व्यवस्था का ढांचा।,The organisational set up of banking in a coun...
2,"दो गवाह जो मारे गए उनमें से एक को तो खुलेआम, द...","Of the two that were killed, one was shot dead..."
3,इस लड़ाई में सारे देश में एक चेतना जागी है।,The whole country has awakened during the war.
4,वह बोले हम तो गुनाहगारों (क़ौमे लूत) की तरफ भे...,"They replied, ""We have been sent to a sinful p..."


In [5]:
df

,hindi,english
0,दो वर्ष की उम्र से स्थायी छेदक दांत पैदा होने ...,"At the age of two years, the permanent incisor..."
1,किसी देश में बैंकिंग व्यवस्था का ढांचा।,The organisational set up of banking in a coun...
2,"दो गवाह जो मारे गए उनमें से एक को तो खुलेआम, द...","Of the two that were killed, one was shot dead..."
3,इस लड़ाई में सारे देश में एक चेतना जागी है।,The whole country has awakened during the war.
4,वह बोले हम तो गुनाहगारों (क़ौमे लूत) की तरफ भे...,"They replied, ""We have been sent to a sinful p..."
...,...,...
99995,और यदि तुम अल्लाह की नेमतों (कृपादानों) को गिन...,"And if you would count Allah 's favors, you wi..."
99996,विश्वविद्यालयों की दो प्रतिष्ठित अंतरराष्ट्रीय...,In two reputed international rankings of unive...
99997,से,From
99998,वह दुःस्वप्न!,That nightmare!


In [6]:
with open('Englishtokenizer.pkl','rb') as f:
    tokenizer = pickle.load(f)
with open('Hinditokenizer.pkl','rb') as f:
    tokenizer_hi = pickle.load(f)

In [7]:
df['english']=df['english'].str.lower()

In [8]:
sequences = tokenizer.texts_to_sequences(df['english'])

In [9]:
from tensorflow.keras.utils import pad_sequences
sequences = pad_sequences(sequences, padding='post', value=0,maxlen=30, truncating='post',)

In [10]:
sequences_hi = tokenizer_hi.texts_to_sequences(df['hindi'])


In [11]:
sequences_hi = pad_sequences(
    sequences_hi,
    maxlen=40,
    padding='post',
    truncating='post',
    value=0
)


In [12]:
len(sequences_hi[0])

40

In [13]:
decoder_input = sequences_hi[:, :-1]
decoder_target = sequences_hi[:, 1:]


In [18]:
embedding_dim = 128
hidden_units = 256
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

encoder_input=Input(shape=(30,))
encoder_embedding=Embedding(30000, embedding_dim)(encoder_input)
encoder_lstm = LSTM(hidden_units, return_sequences=True,return_state=True)
encoder_outputs, state_h,state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h,state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(30000, embedding_dim)(decoder_inputs)

decoder_lstm = LSTM(
    hidden_units,
    return_sequences=True,
    return_state=True
)

decoder_outputs, _, _ = decoder_lstm(
    decoder_embedding,
    initial_state=encoder_states
)
    


In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Softmax
# (batch, Ty, hidden) × (batch, hidden, Tx)
score = tf.matmul(
    decoder_outputs,
    encoder_outputs,
    transpose_b=True
)
# score → (batch, Ty, Tx)

attention_weights = Softmax(axis=-1)(score)
# (batch, Ty, Tx)

context_vector = tf.matmul(attention_weights, encoder_outputs)
# (batch, Ty, hidden)


In [20]:
from tensorflow.keras.layers import Concatenate

decoder_combined_context = Concatenate(axis=-1)(
    [context_vector, decoder_outputs]
)
# (batch, Ty, 2 * hidden)


In [21]:
decoder_dense = Dense(30000, activation='softmax')
decoder_outputs_final = decoder_dense(decoder_combined_context)


In [ ]:
model = Model(
    [encoder_input, decoder_inputs],
    decoder_outputs_final
)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 30, 128)      3840000     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 128)    3840000     ['input_2[0][0]']                
                                                                                              

In [24]:
history = model.fit([sequences, decoder_input], decoder_target, epochs = 10,batch_size=16)

Epoch 1/10
6250/6250 [==============================] - 497s 78ms/step - loss: 1.8626 - accuracy: 0.7388
Epoch 2/10
6250/6250 [==============================] - 433s 69ms/step - loss: 1.4594 - accuracy: 0.7647
Epoch 3/10
6250/6250 [==============================] - 1138s 182ms/step - loss: 1.2260 - accuracy: 0.7831
Epoch 4/10
6250/6250 [==============================] - 358s 57ms/step - loss: 1.0483 - accuracy: 0.7997
Epoch 5/10
6250/6250 [==============================] - 917s 147ms/step - loss: 0.9057 - accuracy: 0.8157
Epoch 6/10
6250/6250 [==============================] - 385s 62ms/step - loss: 0.7893 - accuracy: 0.8320
Epoch 7/10
6250/6250 [==============================] - 12027s 2s/step - loss: 0.6964 - accuracy: 0.8470
Epoch 8/10
6250/6250 [==============================] - 686s 110ms/step - loss: 0.6210 - accuracy: 0.8599
Epoch 9/10
6250/6250 [==============================] - 549s 88ms/step - loss: 0.5595 - accuracy: 0.8712
Epoch 10/10
6250/6250 [============================

In [25]:
model.save("trainingModelwithAttention.h5")

In [35]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 30, 128)      3840000     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        multiple             3840000     ['input_2[0][0]']                
                                                                                              

Inference:

In [28]:
encoder_embedding = model.get_layer("embedding")
encoder_lstm = model.get_layer("lstm")

decoder_embedding = model.get_layer("embedding_1")
decoder_lstm = model.get_layer("lstm_1")

decoder_dense = model.get_layer("dense")


In [29]:
encoder_input = model.input[0]

encoder_embedded = encoder_embedding(encoder_input)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedded)

encoder_model = Model(
    encoder_input,
    [encoder_outputs, state_h, state_c]
)


In [47]:
encoder_model.save("Encoder_inference_withAttention.h5")

In [30]:
decoder_single_input = Input(shape=(1,))
encoder_outputs_inf = Input(shape=(30, hidden_units))
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))


In [31]:
decoder_embedded = decoder_embedding(decoder_single_input)

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedded,
    initial_state=[decoder_state_input_h, decoder_state_input_c]
)


In [32]:
import tensorflow as tf
from tensorflow.keras.layers import Softmax, Concatenate

score = tf.matmul(
    decoder_outputs,
    encoder_outputs_inf,
    transpose_b=True
)

attention_weights = Softmax(axis=-1)(score)

context_vector = tf.matmul(
    attention_weights,
    encoder_outputs_inf
)

decoder_combined_context = Concatenate(axis=-1)(
    [context_vector, decoder_outputs]
)


In [33]:
decoder_outputs_final = decoder_dense(decoder_combined_context)


In [34]:
decoder_model = Model(
    [
        decoder_single_input,
        encoder_outputs_inf,
        decoder_state_input_h,
        decoder_state_input_c
    ],
    [
        decoder_outputs_final,
        state_h,
        state_c
    ]
)


In [48]:
decoder_model.save("Decoder_inference_withAttention.h5")

In [3]:
with open('Englishtokenizer.pkl','rb') as f:
    tokenizer = pickle.load(f)
with open('Hinditokenizer.pkl','rb') as f:
    tokenizer_hi = pickle.load(f)

In [4]:
from tensorflow.keras.models import load_model
encoder_attention=load_model('Encoder_inference_withAttention.h5')
decoder_attention=load_model('Decoder_inference_withAttention.h5')

In [5]:
def decode_sequence(input_seq):
    encoder_outs, h, c = encoder_attention.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer_hi.word_index['<sos>']

    decoded_sentence = []

    for _ in range(40):
        output_tokens, h, c = decoder_attention.predict(
            [target_seq, encoder_outs, h, c],
            verbose=0
        )

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0:
            break
        sampled_word = tokenizer_hi.index_word[sampled_token_index]

        if sampled_word == '<eos>':
            break

        decoded_sentence.append(sampled_word)
        target_seq[0, 0] = sampled_token_index

    return ' '.join(decoded_sentence)


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def clean_sentence(s):
    s = s.lower().strip()
    s=tokenizer.texts_to_sequences([s])
    s=pad_sequences(s,maxlen=30,padding="post")
    return s

In [7]:
check=clean_sentence("I am happy")

In [8]:
decode_sequence(check)

1/1 [==============================] - 1s 1s/step


'खुशी हो रही हूँ।'

In [45]:

decode_sequence(clean_sentence("She is my friend"))

1/1 [==============================] - 0s 18ms/step


'मेरा मित्र है।'

In [9]:
tests = [
    "I went to the market and bought some fruits",
    "She studied hard and passed the exam",
    "He opened the door and walked inside",
    "I finished my work and went home"
]



for t in tests:
    print(t, "→", decode_sequence(clean_sentence(t)))


1/1 [==============================] - 0s 19ms/step
I went to the market and bought some fruits → बाजार से बाजार को सहन करने से मना किया गया, और
1/1 [==============================] - 0s 16ms/step
She studied hard and passed the exam → का अध्ययन <OOV> और <OOV>
1/1 [==============================] - 0s 17ms/step
He opened the door and walked inside → और <OOV> के अंदर जाकर
1/1 [==============================] - 0s 16ms/step
I finished my work and went home → मेरे घर और घर घर लौट आती हूं।
